web scrapping

## RECORDAR QUITAR TODO TIPO DE PUNTUACION QUE LLEVEN LOS NOMBRES

In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.parse
from nltk.sentiment import SentimentIntensityAnalyzer
import re
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from textblob import TextBlob
import csv

# Ruta del archivo CSV con los nombres de los profesores
ruta_archivo_csv = '/home/roy/HORARIO_MAKER/data/ECONOMIA.csv'

# URL base de la página de búsqueda
url_base = 'https://foroupiicsa.net/diccionario/buscar/'

# Crear un diccionario para almacenar los comentarios por profesor
comentarios_por_profesor = {}

# Leer el archivo CSV y crear un DataFrame
data_frame_profesores = pd.read_csv(ruta_archivo_csv)

# Realizar la búsqueda y obtener los comentarios para cada profesor
for index, row in data_frame_profesores.iterrows():
    nombre_profesor = row['Profesor']
    comentarios_profesor = []

    # Escapar los caracteres especiales en el nombre del profesor
    nombre_profesor_url = urllib.parse.quote_plus(nombre_profesor)

    # Construir la URL específica para el profesor
    url_profesor = url_base + nombre_profesor_url

    # Realizar la solicitud GET a la página del profesor
    response = requests.get(url_profesor)
    soup = BeautifulSoup(response.text, 'html.parser')
    comentarios_profesor_html = soup.find_all('p', {'class': 'comentario'})

    # Agregar los comentarios a la lista del profesor actual
    for comentario in comentarios_profesor_html:
        texto_comentario = comentario.text.strip()
        comentarios_profesor.append(texto_comentario)

    # Almacenar los comentarios en el diccionario
    comentarios_por_profesor[nombre_profesor] = comentarios_profesor

# Mostrar los comentarios para cada profesor
for nombre_profesor, comentarios_profesor in comentarios_por_profesor.items():
    print(f"Comentarios para el profesor {nombre_profesor}:")
    for comentario in comentarios_profesor:
        print(comentario)
    print()





Comentarios para el profesor GARCIA LUNA EMMANUEL:
Sean inteligentes y graben las primeras clases en donde el profe les ofrece puntos extra a cambio de $$$$
Te vende un "libro" de 500 por cada alumno, sus examenes son muy perros y ahi cuando el profe les recuerde lo de comprar
el libro para pasar se la voltean y le dicen que lo van a reportar si no le pone 10 a todo el grupo
Es buen profe y la clase no se hace aburrida, pero los exámenes son en equipo y esto es un problema al final; ¿Porque es un problema? Pues te pide una lana acambio de un punto extra por parcial y los que sumen 19 en los primeros parciales los exenta el tercero; entonces como los examenes estan hechos para que justo de tiempo a teminar entre todo tu equipo, si varios en tu equipo excentan, ten por seguro que no vas a terminar el tercer examen tu solo (vale 70%); asi que si cursas con el, comprale su dichoso PDF de $500 para evitarte este problema. SI USTED LEE ESTO CHINGUE A SU MADRE TANTO QUE CRITICA Y ES IGUAL DE 

procesamiento del texto 

In [ ]:
import pandas as pd
import urllib.parse
import requests
import re
import string
from bs4 import BeautifulSoup

# Ruta del archivo CSV con los nombres de los profesores
ruta_archivo_csv = '/home/roy/HORARIO_MAKER/data/ECONOMIA.csv'

# URL base de la página de búsqueda
url_base = 'https://foroupiicsa.net/diccionario/buscar/'

# Crear un diccionario para almacenar los comentarios por profesor
comentarios_por_profesor = {}

# Leer el archivo CSV y crear un DataFrame
data_frame_profesores = pd.read_csv(ruta_archivo_csv)

# Realizar la búsqueda y obtener los comentarios para cada profesor
for index, row in data_frame_profesores.iterrows():
    nombre_profesor = row['Profesor']
    comentarios_profesor = []

    # Escapar los caracteres especiales en el nombre del profesor
    nombre_profesor_url = urllib.parse.quote_plus(nombre_profesor)

    # Construir la URL específica para el profesor
    url_profesor = url_base + nombre_profesor_url

    # Realizar la solicitud GET a la página del profesor
    response = requests.get(url_profesor)
    soup = BeautifulSoup(response.text, 'html.parser')
    comentarios_profesor_html = soup.find_all('p', {'class': 'comentario'})

    # Agregar los comentarios a la lista del profesor actual
    for comentario in comentarios_profesor_html:
        texto_comentario = comentario.text.strip()

        # Convertir a minúsculas
        texto_comentario = texto_comentario.lower()

        # Eliminar signos de puntuación
        texto_comentario = re.sub('[' + string.punctuation + ']', '', texto_comentario)

        # Eliminar fechas en formato YYYY-MM-DD, DD-MM-YYYY, DD/MM/YYYY, DD-MM-YY, DD/MM/YY
        texto_comentario = re.sub(r'\d{4}-\d{2}-\d{2}', '', texto_comentario)
        texto_comentario = re.sub(r'\d{2}-\d{2}-\d{4}', '', texto_comentario)
        texto_comentario = re.sub(r'\d{1,2}/\d{1,2}/\d{4}', '', texto_comentario)
        texto_comentario = re.sub(r'\d{2}/\d{2}/\d{2}', '', texto_comentario)

        comentarios_profesor.append(texto_comentario)

    # Almacenar los comentarios en el diccionario
    comentarios_por_profesor[nombre_profesor] = comentarios_profesor

# Mostrar los comentarios para cada profesor
for nombre_profesor, comentarios_profesor in comentarios_por_profesor.items():
    print(f"Comentarios para el profesor {nombre_profesor}:")
    for comentario in comentarios_profesor:
        print(comentario)
    print()


nltk 

In [22]:
import pandas as pd
import urllib.parse
import requests
import re
import string
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer

# Crear un objeto de SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Ruta del archivo CSV con los nombres de los profesores
ruta_archivo_csv = '/home/roy/HORARIO_MAKER/data/ECONOMIA.csv'

# URL base de la página de búsqueda
url_base = 'https://foroupiicsa.net/diccionario/buscar/'

# Crear un diccionario para almacenar los comentarios, materias y calificaciones por profesor
comentarios_materias_calificaciones = {}

# Leer el archivo CSV y crear un DataFrame
data_frame_profesores = pd.read_csv(ruta_archivo_csv)

# Realizar la búsqueda y obtener los comentarios, materias y fechas para cada profesor
for i, row in data_frame_profesores.iterrows():
    nombre_profesor = row['Profesor']
    comentarios_profesor = []
    materias_profesor = []
    fechas_profesor = []

    # Escapar los caracteres especiales en el nombre del profesor
    nombre_profesor_url = urllib.parse.quote_plus(nombre_profesor)

    # Construir la URL específica para el profesor
    url_profesor = url_base + nombre_profesor_url

    # Realizar la solicitud GET a la página del profesor
    response = requests.get(url_profesor)
    soup = BeautifulSoup(response.text, 'html.parser')
    comentarios_profesor_html = soup.find_all('p', {'class': 'comentario'})
    materias_profesor_html = soup.find_all('span', {'class': 'bluetx negritas'})
    fechas_profesor_html = soup.find_all('p', {'class': 'fecha'})

    # Agregar los comentarios, materias y fechas a las listas del profesor actual
    for comentario in comentarios_profesor_html:
        texto_comentario = comentario.text.strip()
        comentarios_profesor.append(texto_comentario)

    for materia in materias_profesor_html:
        texto_materia = materia.text.strip()
        materias_profesor.append(texto_materia)

    for fecha in fechas_profesor_html:
        texto_fecha = fecha.text.strip()
        fechas_profesor.append(texto_fecha)

    # Realizar el análisis de sentimientos y asignar una puntuación a cada comentario
    puntuaciones_profesor = []
    for comentario in comentarios_profesor:
        sentiment_score = sia.polarity_scores(comentario)
        puntuacion = sentiment_score['compound']
        puntuaciones_profesor.append(puntuacion)

    # Calcular el promedio de las puntuaciones si hay puntuaciones disponibles
    promedio_puntuaciones = sum(puntuaciones_profesor) / len(puntuaciones_profesor) if puntuaciones_profesor else 0.0

    # Almacenar los comentarios, materias, fechas, puntuaciones y promedio en el diccionario
    comentarios_materias_calificaciones[i + 1] = {
        'profesor': nombre_profesor,
        'comentarios': comentarios_profesor,
        'materias': materias_profesor,
        'fechas': fechas_profesor,
        'puntuaciones': puntuaciones_profesor,
        'promedio': promedio_puntuaciones
    }

# Asignar una calificación general a cada profesor
calificaciones_generales = {}
for indice, datos_profesor in comentarios_materias_calificaciones.items():
    promedio_puntuaciones = datos_profesor['promedio']

    if promedio_puntuaciones >= 0.08:
        calificacion_general = 'Muy positivo'
    elif promedio_puntuaciones >= 0.06:
        calificacion_general = 'Positivo'
    elif promedio_puntuaciones >= 0.04:
        calificacion_general = 'Ligeramente positivo'
    elif promedio_puntuaciones >= 0.02:
        calificacion_general = 'Neutro'
    elif promedio_puntuaciones > -0.02:
        calificacion_general = 'Ligeramente negativo'
    elif promedio_puntuaciones > -0.04:
        calificacion_general = 'Negativo'
    elif promedio_puntuaciones > -0.06:
        calificacion_general = 'Muy negativo'
    else:
        calificacion_general = 'Extremadamente negativo'

    calificaciones_generales[indice] = calificacion_general

# Mostrar los comentarios, materias, fechas, puntuaciones y calificaciones generales para cada profesor
for indice, datos_profesor in comentarios_materias_calificaciones.items():
    profesor = datos_profesor['profesor']
    comentarios_profesor = datos_profesor['comentarios']
    materias_profesor = datos_profesor['materias']
    fechas_profesor = datos_profesor['fechas']
    puntuaciones_profesor = datos_profesor['puntuaciones']
    promedio_puntuaciones = datos_profesor['promedio']
    calificacion_general = calificaciones_generales[indice]

    print(f"{indice}) !!PROFESOR!!--> {profesor}:")
    for comentario, materia, fecha, puntuacion in zip(comentarios_profesor, materias_profesor, fechas_profesor, puntuaciones_profesor):
        print(f"Comentario: {comentario}")
        print(f"Materia: {materia}")
        print(f"Fecha: {fecha}")
        print(f"Puntuación de sentimiento: {puntuacion}")
        print()
    print(f"Promedio de puntuaciones: {promedio_puntuaciones}")
    print(f"****CALIFICACION GENERAL****: {calificacion_general}")
    print()


1) !!PROFESOR!!--> GARCIA LUNA EMMANUEL:
Comentario: Sean inteligentes y graben las primeras clases en donde el profe les ofrece puntos extra a cambio de $$$$
Te vende un "libro" de 500 por cada alumno, sus examenes son muy perros y ahi cuando el profe les recuerde lo de comprar
el libro para pasar se la voltean y le dicen que lo van a reportar si no le pone 10 a todo el grupo
Materia: ECONOMÍA
Fecha: 21 Dic 2021
Puntuación de sentimiento: -0.296

Comentario: Es buen profe y la clase no se hace aburrida, pero los exámenes son en equipo y esto es un problema al final; ¿Porque es un problema? Pues te pide una lana acambio de un punto extra por parcial y los que sumen 19 en los primeros parciales los exenta el tercero; entonces como los examenes estan hechos para que justo de tiempo a teminar entre todo tu equipo, si varios en tu equipo excentan, ten por seguro que no vas a terminar el tercer examen tu solo (vale 70%); asi que si cursas con el, comprale su dichoso PDF de $500 para evitart